### 환경 불러오기

In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json
import math
# copywarning ignore
import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from multiprocessing import Process, Value, Array
# 정규분포 함수
from scipy.stats import norm
from itertools import product
from itertools import combinations

### 오피스텔 실거래 데이터 불러오기(2022.03 ~ 2023.02)

In [3]:
# officetels 폴더에 있는 모든 파일명 리스트로 저장
off_forders = os.listdir('../data/officetels')
# csv 파일을 모두 불러와서 병합
df_officetels = pd.DataFrame()

#### 오피스텔 도로명 주소 Geocoding - 위도, 경도 추출

In [4]:
api_key = '79d3d40921cad65ee45ee1059c974972' # daero
# api_key = 'a6bd28c812cc2401914de6d7fcc93171' # jihwan

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['road_address']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

def get_lat_lon(df):
    for i in range(0, len(df)):
        try:
            addr = df.iloc[i]['시군구'] + " " + df.iloc[i]['도로명']
            x, y = addr_to_lat_lon(addr)
        except:
            x, y = 0, 0
        df['longitude'].iloc[i] = x
        df['latitude'].iloc[i] = y
    

### 카카오 rest api로 lat, lon 받아오기

In [5]:
# for i in range(0, len(off_forders)):
#     if off_forders[i].split('.')[1] == 'csv':
#         file = '../data/officetels/'+off_forders[i]
#         # csv파일의 불필요한 주석정보 제거
#         sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
#         sub_df['도로명'].replace(' ',np.nan, inplace=True)
#         sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
#         sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
#         sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
#         get_lat_lon(sub_df)
#         # concat함수로 행을 추가하며 병합
#         sub_df.to_csv(file, encoding='CP949',index=False)
#         df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
#         print(i,'complete')

# df_officetels.to_csv(path_or_buf='오피스텔 실거래가_22-03_23-02', encoding='CP949')

In [6]:
df_officetels = pd.read_csv('../data/오피스텔 실거래가_geo_coding.csv', encoding='CP949')
# 위경도가 주소가 없는 행은 제거
df_officetels = df_officetels[df_officetels['longitude']!=0].reset_index(drop=True)
df_officetels

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_29500\3313208631.py:1: DtypeWarning: Columns (11,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_officetels = pd.read_csv('../data/오피스텔 실거래가_geo_coding.csv', encoding='CP949')


,시군구,법정동,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,...,층,건축년도,도로명,longitude,latitude,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원)
0,강원도 강릉시,교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,11,...,7,2020.0,강릉대로 285,128.899711,37.762775,202203~202303,신규,-,0,0.0
1,강원도 강릉시,교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,19,...,13,2020.0,강릉대로 285,128.899711,37.762775,202206~202305,신규,-,0,0.0
2,강원도 강릉시,교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,23,...,11,2020.0,강릉대로 285,128.899711,37.762775,202203~202304,신규,-,0,0.0
3,강원도 강릉시,교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,25,...,11,2020.0,강릉대로 285,128.899711,37.762775,202204~202404,신규,-,0,0.0
4,강원도 강릉시,교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,27,...,11,2020.0,강릉대로 285,128.899711,37.762775,202205~202305,신규,-,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164182,충청북도 청주흥덕구 오송읍,연제리,664,664,0,두산위브 센티움,월세,28.90,202302,28,...,5,2014.0,오송생명3로 31,127.328279,36.635157,202303~202403,갱신,-,331,43.0
164183,충청북도 청주흥덕구,외북동,317,317,0,아크로폴리스,전세,20.40,202302,4,...,7,2019.0,엘지로 23,127.435104,36.660285,-,-,-,NaN,NaN
164184,충청북도 청주흥덕구,외북동,317,317,0,아크로폴리스,전세,37.05,202302,4,...,5,2019.0,엘지로 23,127.435104,36.660285,202302~202502,신규,-,0,0.0
164185,충청북도 충주시 주덕읍,화곡리,1207,1207,0,시그니처시티,월세,18.82,202302,7,...,10,2020.0,기업도시2로 152,127.816580,37.004184,202302~202402,신규,-,0,0.0


In [7]:
df_hadan = df_officetels[df_officetels['법정동']=='하단동'].reset_index(drop=True)
df_hadan

,시군구,법정동,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,...,층,건축년도,도로명,longitude,latitude,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원)
0,부산광역시 사하구,하단동,517-1,517,1,(517-1),월세,19.95,202203,10,...,6,2021.0,하신번영로300번길 56,128.962117,35.107574,202204~202303,신규,-,0,0.0
1,부산광역시 사하구,하단동,517-1,517,1,(517-1),월세,19.95,202203,14,...,6,2021.0,하신번영로300번길 56,128.962117,35.107574,202203~202303,신규,-,0,0.0
2,부산광역시 사하구,하단동,604-3,604,3,(604-3),월세,16.27,202203,29,...,2,2019.0,동매로23번길 18,128.961233,35.104543,202203~202303,신규,-,0,0.0
3,부산광역시 사하구,하단동,604-8,604,8,(604-8),전세,15.08,202203,26,...,10,2019.0,동매로29번길 17,128.961500,35.104530,-,-,-,NaN,NaN
4,부산광역시 사하구,하단동,618-11,618,11,(618-11),월세,41.44,202203,3,...,11,2021.0,낙동대로457번길 17,128.967929,35.105061,202203~202303,신규,-,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,부산광역시 사하구,하단동,500-12,500,12,현민O/T,월세,14.57,202302,18,...,12,2020.0,낙동대로535번길 45-2,128.962035,35.111025,202302~202402,신규,-,0,0.0
353,부산광역시 사하구,하단동,500-12,500,12,현민O/T,월세,14.57,202302,25,...,8,2020.0,낙동대로535번길 45-2,128.962035,35.111025,202302~202402,신규,-,0,0.0
354,부산광역시 사하구,하단동,500-12,500,12,현민O/T,월세,14.86,202302,26,...,9,2020.0,낙동대로535번길 45-2,128.962035,35.111025,202302~202402,신규,-,0,0.0
355,부산광역시 사하구,하단동,500-12,500,12,현민O/T,월세,14.57,202302,27,...,8,2020.0,낙동대로535번길 45-2,128.962035,35.111025,202302~202402,신규,-,0,0.0


In [8]:
# df_hadan.to_json('../data/json/hadan_dong.json')

In [9]:
# df_officetels.to_json('../data/json/officetels.json')

In [10]:
class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"
    
        # 카테고리 검색 메서드
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
            """
            06 카테고리 검색
            """
            params = {'category_group_code': f"{category_group_code}",
                    'x': f"{x}",
                    'y': f"{y}"}
            
            if radius != None:
                params['radius'] = f"{radius}"
            if rect != None:
                params['rect'] = f"{rect}"
            if page != None:
                params['page'] = f"{page}"
            if size != None:
                params['size'] = f"{size}"
            if sort != None:
                params['sort'] = f"{sort}"
                
            res = requests.get(self.URL_06, headers=self.headers, params=params)
            document = json.loads(res.text)
            
            return document

### 카테고리별 주변 정보 탐색

In [11]:
# REST API 키
# rest_api_key = "79d3d40921cad65ee45ee1059c974972" # Daero
rest_api_key = 'a6bd28c812cc2401914de6d7fcc93171' # Jihwan

kakao = KakaoLocalAPI(rest_api_key)

In [12]:
unique_df_officetels = df_officetels.drop_duplicates('도로명', keep='first', ignore_index=True)
len(unique_df_officetels)

9750

In [13]:
unique_df_officetels_hadan = df_hadan.drop_duplicates('도로명', keep='first', ignore_index=True)
len(unique_df_officetels_hadan)

77

## 매물 주변 정보

### API로 주변정보 검색(병렬처리)

In [14]:
# def get_category(addr, lat, lon):
#     try:
#         # 편의점 : CS2
#         cs2 = kakao.search_category('CS2', lat, lon, 300)
#         # 학교 : SC4
#         sc4 = kakao.search_category('SC4', lat, lon, 300)
#         # 지하철역 : SW8
#         sw8 = kakao.search_category('SW8', lat, lon, 300)
#         # 음식점 : FD6
#         fd6 = kakao.search_category('FD6', lat, lon, 300)
#         # 카페 : CE7
#         ce7 = kakao.search_category('CE7', lat, lon, 300)
#         # 병원 : HP8
#         hp8 = kakao.search_category('HP8', lat, lon, 300)
#         # 약국 : PM9
#         pm9 = kakao.search_category('PM9', lat, lon, 300)

#         category = {"CS2": [cs2["documents"]], "SC4": [sc4["documents"]], "SW8": [sw8["documents"]], "FD6": [fd6["documents"]], "CE7": [ce7["documents"]], "HP8": [hp8["documents"]], "PM9": [pm9["documents"]]}

#         around_data_json['officetels'].append({
#             "address": addr,
#             "latitude": lat,
#             "longitude": lon,
#             "category": [category]
#         })
#         print(category) 
#     except:     
#         print(addr)
        

# around_data_json = {}
# around_data_json['officetels'] = []

# addr = unique_df_officetels['도로명'].to_list()
# lat = unique_df_officetels['latitude'].to_list()
# lon = unique_df_officetels['longitude'].to_list()

# process = Process(target=get_category, args=(addr, lat, lon)) 
# process.start()
# process.join()

# around_data_json


### API로 주변정보 검색

In [15]:
def search_x_y(series):
    addr = series['시군구'] + " " + series['법정동'] + " " + series['번지']
    dong = series['법정동']
    lon = series['longitude']
    lat = series['latitude']

    # 편의점 : CS2
    try:
        cs2_documnets = kakao.search_category('CS2', lon, lat, radius=300, page=1, size=15)
        cs2_pages = math.ceil(cs2_documnets['meta']['pageable_count'] / 15)
        cs2_total = cs2_documnets['meta']['total_count']
        cs2 = cs2_documnets["documents"]
    except:
        cs2_documnets = {}
        cs2_pages = 0
        cs2_total = 0
        cs2 = {}
    # 학교 : SC4
    try:
        sc4_documnets = kakao.search_category('SC4', lon, lat, radius=300, page=1, size=15)
        sc4_pages =  math.ceil(sc4_documnets['meta']['pageable_count'] / 15)
        sc4_total = sc4_documnets['meta']['total_count']
        sc4 = sc4_documnets["documents"]
    except:
        sc4_documnets = {}
        sc4_pages = 0
        sc4_total = 0
        sc4 = {}
    # 지하철역 : SW8
    try:
        sw8_documnets = kakao.search_category('SW8', lon, lat, radius=300, page=1, size=15)
        sw8_pages =  math.ceil(sw8_documnets['meta']['pageable_count'] / 15)
        sw8_total = sw8_documnets['meta']['total_count']
        sw8 = sw8_documnets["documents"]
    except:
        sw8_documnets = {}
        sw8_pages = 0
        sw8_total = 0
        sw8 = {}
    # 음식점 : FD6
    try:
        fd6_documnets = kakao.search_category('FD6', lon, lat, radius=300, page=1, size=15)
        fd6_pages = math.ceil(fd6_documnets['meta']['pageable_count'] / 15)
        fd6_total = fd6_documnets['meta']['total_count']
        fd6 = fd6_documnets["documents"]
    except:
        fd6_documnets = {}
        fd6_pages = 0
        fd6_total = 0
        fd6 = {}
    # 카페 : CE7
    try:
        ce7_documnets = kakao.search_category('CE7', lon, lat, radius=300, page=1, size=15)
        ce7_pages =  math.ceil(ce7_documnets['meta']['pageable_count'] / 15)
        ce7_total = ce7_documnets['meta']['total_count']
        ce7 = ce7_documnets["documents"]
    except:
        ce7_documnets = {}
        ce7_pages = 0
        ce7_total = 0
        ce7 = {}
    # 병원 : HP8
    try:
        hp8_documnets = kakao.search_category('HP8', lon, lat, radius=300, page=1, size=15)
        hp8_pages =  math.ceil(hp8_documnets['meta']['pageable_count'] / 15)
        hp8_total = hp8_documnets['meta']['total_count']
        hp8 = hp8_documnets["documents"]
    except:
        hp8_documnets = {}
        hp8_pages = 0
        hp8_total = 0
        hp8 = {}
    # 약국 : PM9
    try:
        pm9_documnets = kakao.search_category('PM9', lon, lat, radius=300, page=1, size=15)
        pm9_pages =  math.ceil(pm9_documnets['meta']['pageable_count'] / 15)
        pm9_total = pm9_documnets['meta']['total_count']
        pm9 = pm9_documnets["documents"]
    except:
        pm9_documnets = {}
        pm9_pages = 0
        pm9_total = 0
        pm9 = {}

    if cs2_pages > 1:
        for i in range(2, cs2_pages + 1):
            cs2 = cs2 + kakao.search_category('CS2', lon, lat, radius=300, page=i, size=15)["documents"]
    if sc4_pages > 1:
        for i in range(2, sc4_pages + 1):
            sc4 = sc4 + kakao.search_category('SC4', lon, lat, radius=300, page=i, size=15)["documents"]
    if sw8_pages > 1:
        for i in range(2, sw8_pages + 1):
            sw8 = sw8 + kakao.search_category('SW8', lon, lat, radius=300, page=i, size=15)["documents"]
    if fd6_pages > 1:
        for i in range(2, fd6_pages + 1):
            fd6 = fd6 + kakao.search_category('FD6', lon, lat, radius=300, page=i, size=15)["documents"]
    if ce7_pages > 1:
        for i in range(2, ce7_pages + 1):
            ce7 = ce7 + kakao.search_category('CE7', lon, lat, radius=300, page=i, size=15)["documents"]      
    if hp8_pages > 1:
        for i in range(2, hp8_pages + 1):
            hp8 = hp8 + kakao.search_category('HP8', lon, lat, radius=300, page=i, size=15)["documents"]
    if pm9_pages > 1:
        for i in range(2, pm9_pages + 1):
            pm9 = pm9 + kakao.search_category('PM9', lon, lat, radius=300, page=i, size=15)["documents"]

    category = {"CS2": cs2, "SC4": sc4, "SW8": sw8, "FD6": fd6, "CE7": ce7, "HP8": hp8, "PM9": pm9}

    dic = {
        "address": addr,
        "dong": dong,
        "x": lon,
        "y": lat,
        "cs2_total": cs2_total,
        "sc4_total": sc4_total,
        "sw8_total": sw8_total,
        "fd6_total": fd6_total,
        "ce7_total": ce7_total,
        "hp8_total": hp8_total,
        "pm9_total": pm9_total,
        "category": [category]
    }
    return dic


#### 전국 매물 주변 정보 검색

In [16]:
# REST API 키
# rest_api_key = "79d3d40921cad65ee45ee1059c974972" # Daero
rest_api_key = 'a6bd28c812cc2401914de6d7fcc93171' # Jihwan

kakao = KakaoLocalAPI(rest_api_key)

In [17]:
# around_data_json = {}
# around_data_json['officetels'] = []

# for i in range(0, len(unique_df_officetels)):
#     off = unique_df_officetels.iloc[i]
#     around_x_y = search_x_y(off)
#     around_data_json['officetels'].append(around_x_y)

#### 하단동 매물 주변 정보 검색

In [18]:
# around_data_hadan_json = {}
# around_data_hadan_json['officetels'] = []

# for i in range(0, len(unique_df_officetels_hadan)):
#     off = unique_df_officetels_hadan.iloc[i]
#     around_x_y = search_x_y(off)
#     around_data_hadan_json['officetels'].append(around_x_y)

#### 파일 저장(json)

In [19]:
# json_pile_path = '../data/json/around_data.json'
# with open(json_pile_path, 'w') as f:
#     json.dump(around_data_json, f)

In [20]:
# json_pile_path = '../data/json/around_data_hadan.json'
# with open(json_pile_path, 'w') as f:
#     json.dump(around_data_hadan_json, f)

### DataFrame으로 주변정보 불러오기

#### 하단동 주변정보 DataFrame

In [21]:
# 주변정보 데이터
df = pd.read_json('../data/json/around_data_hadan.json', orient = "values")
around_hadan_df = pd.DataFrame(df['officetels'].to_list())
around_hadan_df.set_index('address')

,dong,x,y,cs2_total,sc4_total,sw8_total,fd6_total,ce7_total,hp8_total,pm9_total,category
address,,,,,,,,,,,
부산광역시 사하구 하단동 517-1,하단동,128.962117,35.107574,11,0,0,85,16,15,5,[{'CS2': [{'address_name': '부산 사하구 하단동 517-35'...
부산광역시 사하구 하단동 604-3,하단동,128.961233,35.104543,6,0,0,88,15,11,3,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',..."
부산광역시 사하구 하단동 604-8,하단동,128.961500,35.104530,6,0,0,89,14,12,3,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',..."
부산광역시 사하구 하단동 618-11,하단동,128.967929,35.105061,12,1,1,265,37,70,14,[{'CS2': [{'address_name': '부산 사하구 하단동 589-12'...
부산광역시 사하구 하단동 900-12,하단동,128.960179,35.104186,6,0,0,82,13,2,0,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',..."
...,...,...,...,...,...,...,...,...,...,...,...
부산광역시 사하구 하단동 500-9,하단동,128.963581,35.110615,13,0,0,301,33,2,1,[{'CS2': [{'address_name': '부산 사하구 하단동 496-16'...
부산광역시 사하구 하단동 500-49,하단동,128.963095,35.110839,12,1,0,292,31,2,1,"[{'CS2': [{'address_name': '부산 사하구 하단동 800-9',..."
부산광역시 사하구 하단동 500-81,하단동,128.963292,35.110992,13,1,0,303,32,2,1,"[{'CS2': [{'address_name': '부산 사하구 하단동 800-9',..."


In [22]:
address = df_officetels['시군구'] + " " + df_officetels['법정동'] + " " + df_officetels['번지']
df_officetels.insert(0, 'address', address)
df_officetels.drop(['시군구', '법정동', '번지'] ,axis=1,inplace=True)
df_officetels.set_index('address')

,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,longitude,latitude,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원)
address,,,,,,,,,,,,,,,,,,,
강원도 강릉시 교동 140-16,140,16,강릉역 블루핀,월세,22.00,202203,11,400,50,7,2020.0,강릉대로 285,128.899711,37.762775,202203~202303,신규,-,0,0.0
강원도 강릉시 교동 140-16,140,16,강릉역 블루핀,월세,22.00,202203,19,500,43,13,2020.0,강릉대로 285,128.899711,37.762775,202206~202305,신규,-,0,0.0
강원도 강릉시 교동 140-16,140,16,강릉역 블루핀,월세,22.00,202203,23,50,38,11,2020.0,강릉대로 285,128.899711,37.762775,202203~202304,신규,-,0,0.0
강원도 강릉시 교동 140-16,140,16,강릉역 블루핀,월세,22.00,202203,25,500,47,11,2020.0,강릉대로 285,128.899711,37.762775,202204~202404,신규,-,0,0.0
강원도 강릉시 교동 140-16,140,16,강릉역 블루핀,월세,22.00,202203,27,500,50,11,2020.0,강릉대로 285,128.899711,37.762775,202205~202305,신규,-,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
충청북도 청주흥덕구 오송읍 연제리 664,664,0,두산위브 센티움,월세,28.90,202302,28,450,43,5,2014.0,오송생명3로 31,127.328279,36.635157,202303~202403,갱신,-,331,43.0
충청북도 청주흥덕구 외북동 317,317,0,아크로폴리스,전세,20.40,202302,4,"5,000",0,7,2019.0,엘지로 23,127.435104,36.660285,-,-,-,NaN,NaN
충청북도 청주흥덕구 외북동 317,317,0,아크로폴리스,전세,37.05,202302,4,"10,000",0,5,2019.0,엘지로 23,127.435104,36.660285,202302~202502,신규,-,0,0.0


In [23]:
# 실거래가 데이터
df_total = pd.merge(around_hadan_df, df_officetels, left_index=True, right_index=True, how='inner')
df_total.rename(columns={'address_x':'address'}, inplace=True)
df_total.drop(['address_y','본번','부번','계약일','계약년월','도로명', 'longitude', 'latitude', '계약기간', '계약구분', '갱신요구권 사용'], axis=1, inplace=True)
df_total

,address,dong,x,y,cs2_total,sc4_total,sw8_total,fd6_total,ce7_total,hp8_total,...,category,단지명,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,건축년도,종전계약 보증금 (만원),종전계약 월세 (만원)
0,부산광역시 사하구 하단동 517-1,하단동,128.962117,35.107574,11,0,0,85,16,15,...,[{'CS2': [{'address_name': '부산 사하구 하단동 517-35'...,강릉역 블루핀,월세,22.00,400,50,7,2020.0,0,0.0
1,부산광역시 사하구 하단동 604-3,하단동,128.961233,35.104543,6,0,0,88,15,11,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',...",강릉역 블루핀,월세,22.00,500,43,13,2020.0,0,0.0
2,부산광역시 사하구 하단동 604-8,하단동,128.961500,35.104530,6,0,0,89,14,12,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',...",강릉역 블루핀,월세,22.00,50,38,11,2020.0,0,0.0
3,부산광역시 사하구 하단동 618-11,하단동,128.967929,35.105061,12,1,1,265,37,70,...,[{'CS2': [{'address_name': '부산 사하구 하단동 589-12'...,강릉역 블루핀,월세,22.00,500,47,11,2020.0,0,0.0
4,부산광역시 사하구 하단동 900-12,하단동,128.960179,35.104186,6,0,0,82,13,2,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',...",강릉역 블루핀,월세,22.00,500,50,11,2020.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,부산광역시 사하구 하단동 500-9,하단동,128.963581,35.110615,13,0,0,301,33,2,...,[{'CS2': [{'address_name': '부산 사하구 하단동 496-16'...,남춘천역 코아루웰라움타워,전세,21.38,"13,500",0,22,2020.0,0,0.0
73,부산광역시 사하구 하단동 500-49,하단동,128.963095,35.110839,12,1,0,292,31,2,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 800-9',...",남춘천역 코아루웰라움타워,월세,26.21,"1,000",42,11,2020.0,"1,000",40.0
74,부산광역시 사하구 하단동 500-81,하단동,128.963292,35.110992,13,1,0,303,32,2,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 800-9',...",남춘천역 코아루웰라움타워,월세,41.44,"3,000",45,4,2020.0,0,0.0
75,부산광역시 사하구 하단동 870-193,하단동,128.960780,35.106926,8,0,0,68,12,6,...,"[{'CS2': [{'address_name': '부산 사하구 하단동 604-8',...",남춘천역 코아루웰라움타워,월세,49.91,"2,000",70,5,2020.0,0,0.0


In [24]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        77 non-null     object 
 1   dong           77 non-null     object 
 2   x              77 non-null     float64
 3   y              77 non-null     float64
 4   cs2_total      77 non-null     int64  
 5   sc4_total      77 non-null     int64  
 6   sw8_total      77 non-null     int64  
 7   fd6_total      77 non-null     int64  
 8   ce7_total      77 non-null     int64  
 9   hp8_total      77 non-null     int64  
 10  pm9_total      77 non-null     int64  
 11  category       77 non-null     object 
 12  단지명            77 non-null     object 
 13  전월세구분          77 non-null     object 
 14  전용면적(㎡)        77 non-null     float64
 15  보증금(만원)        77 non-null     object 
 16  월세(만원)         77 non-null     object 
 17  층              77 non-null     int64  
 18  건축년도        

## 상위요소별 dataframe 나누기

#### 변수들의 확률 구하기

In [29]:
## PDF : 확률 질량 함수
def get_prob_by_norm(df, col, index):
    prob = norm(df[col].mean(),df[col].std()).cdf(df[col].iloc[index])
    return prob

## 기준값 얻기 (25%)
def get_standard(df, cols):
    standards = {}
    for col in cols:
        stdr = np.percentile(df[col].values, 25)
        standards[col] = stdr
    return standards

## 관측값이 기준값을 넘을 확률 계산
def trans_prob_by_standard(df, col, standard):
    prob = len(df[df[col]>standard]) / len(df[col])
    return prob

## 관측값을 통해 기준값을 넘는지 안넘는지 evemt 판별
def observe_event(df, col, standard, idx):
    event = (df[col].iloc[idx] >= standard)
    return event

## 하위요인별 관측값이 기준값 보다 높을 확률 계산
def get_criteria_prob(df, cols):
    probs = {}
    standards = get_standard(df, cols)
    for col in cols:
        prob = trans_prob_by_standard(df, col, standards[col])
        probs[col] = prob
    return probs

In [34]:
def make_dataframe(main_df, criteria, cols):
    df = main_df[['address','dong','x','y']+cols].copy()
    df.insert(len(df.columns), 'observed_{}'.format(criteria), [np.NaN for i in range(0, len(df))])
    for i in range(0, len(df)):
        events = {}
        standards = get_standard(df, cols)
        for col in cols:
            event = observe_event(df, col, standards[col], i)
            events[col] = event
        df['observed_{}'.format(criteria)].at[i] = events
    df.insert(len(df.columns),'class', np.random.choice([True,False],size=len(df)))
    return df

In [47]:
# 상위요인명
criteria = ['safety','economic','convenience','traffic']

# 상위요인의 하위 변수 컬럼명
cols_safety = ['cctv','police_office','fire_station','emrg_bell','drunk_store']
cols_economic = ['money']
cols_convenience = ['cs2_total','fd6_total','ce7_total','hp8_total','pm9_total']
cols_traffic = ['bus_stop','sw8_total']

# for cri in criteria:
#     globals()['df_{}'.format(cri)] = make_dataframe(df_total, cri, globals()['cols_{}'.format(cri)])
globals()['df_{}'.format('convenience')] = make_dataframe(df_total, 'convenience', globals()['cols_{}'.format('convenience')])

KeyError: 'cols_events'

In [36]:
df_convenience

,address,dong,x,y,cs2_total,fd6_total,ce7_total,hp8_total,pm9_total,observed_convenience,class
0,부산광역시 사하구 하단동 517-1,하단동,128.962117,35.107574,11,85,16,15,5,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",True
1,부산광역시 사하구 하단동 604-3,하단동,128.961233,35.104543,6,88,15,11,3,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",True
2,부산광역시 사하구 하단동 604-8,하단동,128.961500,35.104530,6,89,14,12,3,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",True
3,부산광역시 사하구 하단동 618-11,하단동,128.967929,35.105061,12,265,37,70,14,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",False
4,부산광역시 사하구 하단동 900-12,하단동,128.960179,35.104186,6,82,13,2,0,"{'cs2_total': True, 'fd6_total': False, 'ce7_t...",True
...,...,...,...,...,...,...,...,...,...,...,...
72,부산광역시 사하구 하단동 500-9,하단동,128.963581,35.110615,13,301,33,2,1,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",False
73,부산광역시 사하구 하단동 500-49,하단동,128.963095,35.110839,12,292,31,2,1,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",True
74,부산광역시 사하구 하단동 500-81,하단동,128.963292,35.110992,13,303,32,2,1,"{'cs2_total': True, 'fd6_total': True, 'ce7_to...",True
75,부산광역시 사하구 하단동 870-193,하단동,128.960780,35.106926,8,68,12,6,2,"{'cs2_total': True, 'fd6_total': False, 'ce7_t...",True


In [40]:
df_convenience['observed_convenience'][0]

{'cs2_total': True,
 'fd6_total': True,
 'ce7_total': True,
 'hp8_total': True,
 'pm9_total': True}

In [41]:
get_criteria_prob(df_convenience, cols_convenience)

{'cs2_total': 0.7012987012987013,
 'fd6_total': 0.7272727272727273,
 'ce7_total': 0.7012987012987013,
 'hp8_total': 0.6753246753246753,
 'pm9_total': 0.6493506493506493}

## BN

### 베이지안 네트워크 라이브러리 호출

In [42]:
import pgmpy as pg
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph

from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph

c:\Users\SAMSUNG\anaconda3\envs\GVCS\lib\site-packages\pgmpy\global_vars.py:11: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device = torch.device("cpu")


### 조건부확률 얻기

In [43]:
def conditional_prob(df, cols):
    bool_val = [True,False]
    result = []
    combines = [list(a) for a in product(bool_val, repeat=len(cols))]
    for combine in combines:
        a_and_b = 0
        b = 0
        for i in range(0, len(df)):
            if list(df['observed_event'].iloc[i].values()) == combine:
                b += 1
                if df['class'].iloc[i] == True:
                    a_and_b += 1
        if(b == 0):
            result.append(0)
        else:
            result.append(a_and_b / b)

    return result   # size : 2^len(cols)

## BN 모델구축

In [61]:
class BN:
    def __init__(self):
        self.criteria = ['safety','economic','convenience','traffic']
        self.cols_safety = ['cctv','police_office','fire_station','emrg_bell','drunk_store']
        self.cols_economic = ['money']
        self.cols_convenience = ['cs2_total','fd6_total','ce7_total','hp8_total','pm9_total']
        self.cols_traffic = ['bus_stop','sw8_total']

    def get_criteria_prob():
        a = 0

    def conditional_prob(self, df, cri, cols):
        bool_val = [True,False]
        result = []
        combines = [list(a) for a in product(bool_val, repeat=len(cols))]
        for combine in combines:
            a_and_b = 0
            b = 0
            for i in range(0, len(df)):
                if list(df['observed_event'].iloc[i].values()) == list(combine):
                    b += 1
                    if df['target_'+cri].iloc[i] == True:
                        a_and_b += 1
            if(b == 0):
                result.append(0)
            else:
                result.append(a_and_b / b)

        return result   # size : 2^len(cols)

    def create_model(self):
        for cri in self.criteria:
            globals()['self.{}_model'.format(cri)] = BayesianModel([tuple(p) for p in product(globals()['self.cols_{}'.format(cri)],[cri], repeat=1)])


In [62]:
print(conditional_prob(around_hadan_df, cols_safety))

KeyError: 'observed_event'

In [63]:
a = [True, False]
b = ['A','B','C']
events = {'a':True, 'b':True, 'c':False}
combines = [tuple(i) for i in product(a, repeat=len(b))]
for com in combines:
    if list(events.values()) == list(com):
        print(list(events.values()), list(com))

[True, True, False] [True, True, False]


## HMM